In [1]:
lines = spark.read.csv('train.csv', header = False)

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
956,application_1551670200070_0574,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


In [2]:
lines.count()

80419

In [3]:
lines.take(3)

[Row(_c0='1', _c1='1', _c2='4.0', _c3='964982703'), Row(_c0='1', _c1='3', _c2='4.0', _c3='964981247'), Row(_c0='1', _c1='6', _c2='4.0', _c3='964982224')]

In [4]:
rdd = sc.parallelize(lines.take(80419)) # 80419
rdd.take(2)

[Row(_c0='1', _c1='1', _c2='4.0', _c3='964982703'), Row(_c0='1', _c1='3', _c2='4.0', _c3='964981247')]

In [40]:
mean_ri = (rdd  # -->(user ID, movie ID, rate, time)
           
           .map(lambda x: (int(x[1]), int(x[0]), float(x[2]))) # -->(movie ID, user ID, rate)
           .sortBy(lambda x: x[0]) # sort by movie ID 
           .map(lambda x: (x[0], (x[2], 1) )) # -->(movie ID, (rate, 1))
           .reduceByKey(lambda x, y: (x[0]+y[0], x[1]+y[1]) ) # -->(movie ID, (total rate, times))
           .sortBy(lambda x: x[0]) # sort by movie ID 
           .map(lambda x: (x[0], x[1][0]/x[1][1])) # -->(movie ID, mean)
            
            )

In [158]:
mean_list = mean_ri.take(mean_ri.count()) 
mean_list[:3]

[(1, 3.9209302325581397), (2, 3.4318181818181817), (3, 3.5384615384615383)]

In [133]:
def splitByKey(x): # reverse function of groupByKey. use key to split every element in value 
    list_ = []
    key_ = x[0]
    for i in x[1]:
        list_.append((key_, i))
    return list_


def fun_minus(x, mean_list): # every rate value minus mean in this row. So we can get r_is - r_i

    mean_ = -1
    for i in mean_list: # find mean in x[0]th row 
        if x[0] == i[0]:
            mean_ = i[1]
            break 

    for i in range(len(x[1])):

        x[1][i] = (x[1][i][0], x[1][i][1]-mean_)
    return x 
        
# x = (1, [(328, 5.0), (330, 4.0), (332, 4.0), (334, 3.5)])
# fun_minus(x, mean_list)
# splitByKey(x)

[(1, (328, 1.0790697674418603)), (1, (330, 0.07906976744186034)), (1, (332, 0.07906976744186034)), (1, (334, -0.42093023255813966))]

In [149]:
minus_mean = (rdd  # -->(user ID, movie ID, rate, time)
             .map(lambda x: (int(x[1]), (int(x[0]), float(x[2]))))  # -->(movie ID, (user ID, rate)  )
             .groupByKey()
             .sortByKey()
             .map(lambda x: (x[0], list(x[1])))  # -->(movie ID, [(user ID, rate), ...]  )
             .map(lambda x: fun_minus(x, mean_list))  # -->(movie ID, [(user ID, rate-mean), ...]  )
             .flatMap(splitByKey)  # -->(movie ID, (user ID, rate-mean))
             .map(lambda x: (x[1][0], x[0], x[1][1]))  # -->(user ID, movie ID, rate)
             .sortBy(lambda x: x[1])
             .sortBy(lambda x: x[0])
#              .sortBy(lambda x: x[1])
#              .sortBy(lambda x: x[0])
            )

In [179]:
minus_mean.take(2)


[(1, 3489, 0.641509433962264), (1, 3527, 0.41803278688524603)]

In [176]:
import itertools as it
import math 

In [163]:
i_j_ri_rj = (minus_mean  # -->(user ID, movie ID, rate)
             .map(lambda x: (int(x[0]), (int(x[1]), float(x[2]))))  # -->(user ID, (movie ID, rate)  )
             .groupByKey()
             .sortByKey()
             .map(lambda x: list(x[1]))  # -->[(movie ID_1, rate_1), ...]
             .map(lambda x: list( it.combinations(sorted(x, key=lambda x: x[0]), 2))) # need to be ordered 
             .flatMap(lambda x: x)  # -->((1, 4.0), (3, 4.0))
             .map(lambda x: ((x[0][0], x[1][0]),(x[0][1], x[1][1]))       )   # -->(  (movie ID_1, ID_2), (rate_1, rate_2)   ) 
#              .filter(lambda x: x[0][1] < x[0][0])    

            
            )

In [164]:
i_j_ri_rj.take(5)

[((1, 3), (0.07906976744186034, 0.4615384615384617)), ((1, 6), (0.07906976744186034, 0.0)), ((1, 47), (0.07906976744186034, 1.0350000000000001)), ((1, 50), (0.07906976744186034, 0.6944444444444446)), ((1, 70), (0.07906976744186034, -0.5090909090909093))]

In [174]:
similarity = (i_j_ri_rj
              .map(lambda x: (x[0], (x[1][0]* x[1][1], x[1][0]**2, x[1][1]**2))  )# -->(  (i, j), (r_i*r_j, r_i**2, r_j**2)   ) 
              .reduceByKey(lambda x, y: (x[0]+y[0], x[1]+y[1], x[2]+y[2]) )


            )

In [175]:
similarity.take(2)

[((10, 1374), (7.4415933528836735, 10.454731978879705, 22.26899063475546)), ((318, 1704), (3.1038836195847184, 19.44429525330236, 30.14507570041749))]

In [177]:
similarity_1 = (similarity.map(lambda x: (x[0], x[1][0]/(math.sqrt(x[1][1])* math.sqrt(x[1][2])    )))

)

In [178]:
similarity_1.take(2)

[((1097, 2985), 0.07485762487258603), ((1517, 2657), 0.29785283807225976)]